In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2 as cv
import sys
import pickle

In [2]:
SR_file_path = '/kaggle/input/imd-gridded-vanilla/data/data.pkl'
with open(SR_file_path, 'rb') as file:
    SR_data = pickle.load(file)

In [3]:
SR_data_ = []
for key in SR_data:
    for j in range(SR_data[key].shape[0]):
        k = SR_data[key][j][1:,:]
        k = k[:,3:-4]
        SR_data_.append(k)

In [4]:
daily_max = np.nanmax(SR_data_, axis=(1, 2))
for i in range(len(SR_data_)):
    max_value = np.nanmax(SR_data_[i])
    if max_value != 0:
        SR_data_[i]/=max_value
SR_data_ = np.nan_to_num(SR_data_, nan=0)

In [5]:
latent_dim = 256

# Encoder
input_layer = layers.Input(shape=(128, 128, 1))
x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(input_layer)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)

x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)


x = layers.Flatten()(x)

# Latent space representation
z = layers.Dense(latent_dim, name='latent_space')(x)

# Decoder
x = layers.Dense(8 * 8 * 512, activation='relu')(z)
x = layers.Reshape((8, 8, 512))(x)

x = layers.Conv2DTranspose(512, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(256, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(128, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)

x = layers.Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2DTranspose(1, (3, 3), padding='same', activation='relu')(x)
autoencoder = models.Model(input_layer, decoded, name="autoencoder")
# def ssim_loss(y_true, y_pred):
#     return 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
def combined_loss(y_true, y_pred):
    ssim_loss = 1 - tf.reduce_mean(tf.image.ssim(y_true, y_pred, max_val=1.0))
    mse_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    return 0.9 * ssim_loss + 0.1* mse_loss
autoencoder.compile(optimizer='adam', loss=combined_loss,metrics = ['mse'])
autoencoder.summary()

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64, 64, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32, 32, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 16, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent_space (Dense)            │ (None, 256)            │     8,388,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32768)          │     8,421,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 8, 8, 512)      │     2,359,808 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 8, 8, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 16, 16, 256)    │     1,179,904 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 22,276,865 (84.98 MB)

 Trainable params: 22,273,025 (84.96 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [6]:
autoencoder.fit(SR_data_,SR_data_,epochs=20,batch_size=32)

Epoch 1/20


I0000 00:00:1727610490.722776     104 service.cc:145] XLA service 0x7dd2fc022040 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727610490.722834     104 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
W0000 00:00:1727610491.570875     104 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert/Assert
W0000 00:00:1727610491.571893     104 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_1/Assert
W0000 00:00:1727610491.572757     104 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_2/Assert
W0000 00:00:1727610491.573371     104 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_3/Assert


   2/1404 ━━━━━━━━━━━━━━━━━━━━ 1:14 53ms/step - loss: 1.0284 - mse: 1.2971  

I0000 00:00:1727610499.906053     104 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1403/1404 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.2239 - mse: 0.0579

W0000 00:00:1727610565.282396     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert/Assert
W0000 00:00:1727610565.282641     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_1/Assert
W0000 00:00:1727610565.283086     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_2/Assert
W0000 00:00:1727610565.283358     105 assert_op.cc:38] Ignoring Assert operator compile_loss/combined_loss/SSIM/Assert_3/Assert


1404/1404 ━━━━━━━━━━━━━━━━━━━━ 88s 51ms/step - loss: 0.2238 - mse: 0.0578
Epoch 2/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.1057 - mse: 0.0052
Epoch 3/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 47ms/step - loss: 0.0863 - mse: 7.7482e-04
Epoch 4/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0942 - mse: 8.0142e-04
Epoch 5/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0781 - mse: 6.4874e-04
Epoch 6/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0733 - mse: 6.0951e-04
Epoch 7/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0696 - mse: 5.7542e-04
Epoch 8/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0697 - mse: 5.7541e-04
Epoch 9/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0653 - mse: 5.3207e-04
Epoch 10/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0631 - mse: 5.1294e-04
Epoch 11/20
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 65s 46ms/step - loss: 0.0613 - mse: 4.9768e-04
Epoch 12/20
1404/1404 ━━━━━━━━━━━━━━━━

In [7]:
autoencoder.save('autoencoder_fe.keras')